In [3]:
import re
import json

input_file = 'regs_sb7900_riscv.sinc'
output_file = 'regs_sb7900_riscv.json'


register_count = 0

def find(pattern, string):
    try:
        result = re.search(pattern, string)
        return result.group().strip('"')
    except:
        ###print("[Pattern not found: ]", pattern, string[0:30])
        raise

def parse_bits(root, data):
    bit_dic = {}
    for bc in data:
        #### Bit Name
        bit = find(r'(?<=Bits )"(\w+)"', bc)
        root[bit] = {}

        #### Bit Position
        position = find(r'(?<=Position: )(.*)', bc)
        position_list = [int(x) for x in position.split("..")]
        position_list.reverse()
        root[bit]["Position"] = position_list
        
        #### Bit Type
        root[bit]["Type"] = find(r'(?<=Type:     )"(\w+)"', bc)
        
        #### Bit Reset
        try:
            root[bit]["Reset"] = find(r'(?<=Reset:    )(\w+)', bc)
        except:
            pass
        
        #### Bit Comment
        try:
            root[bit]["Comment"] = find(r'(?<=Comment:  )(.*)', bc)
        except:
            pass
        
        #### Bit Defines
        try:
            root[bit]["Defines"] = find(r'(?<=Defines:  )(.*)', bc)
        except:
            pass

        print("\t\tBit: ", bit, root[bit]["Position"])

def parse_registers(root, data):

    pattern = r'  Register .*(?:\n|.)*?\}\n\n'
    registers = re.findall(pattern, data, re.MULTILINE)
    
    for r in registers:
        
        #### Register Name
        pattern = r'(?<=  Register )"(\w+)"'
        register = find(pattern, r)
        
        root[register] = {}

        #### Register Offset
        root[register]["Offset"] = int(find(r'(?<=Offset:  )(\w+)', r), 16)

        #### Register Description
        try:
            root[register]["Description"] = find(r'(?<=Description: )"(.*)"', r)
        except:
            pass

        #### Register Defines
        try:
            root[register]["Defines"] = find(r'(?<=Defines: )"(.*)"', r)
        except:
            pass

        #### Register Read Mask
        root[register]["Read Mask"] = find(r'(?<=Read Mask:   )(\w+)', r)
        
        #### Register Write Mask
        root[register]["Write Mask"] = find(r'(?<=Write Mask:  )(\w+)', r)
        
        #### Register Reset Mask
        try:
            root[register]["Reset Value"] = find(r'(?<=Reset Value: )(\w+)', r)
        except:
            pass
        
        bits_content = re.findall(r'\n    Bits.*(?:\n|.)*?\}', r)
        
        root[register]["Bits"] = {}
        
        print("\t[Register]", register)
        parse_bits(root[register]["Bits"], bits_content)
        
        global register_count
        register_count = register_count + 1

def parse_categories(root, data):
    ###pattern = r'(?<=;; ========= )(.*)(?= ========)'
    pattern = r'Block .*(?:\n|.)*?\n\}\n'
    categories = re.findall(pattern, data, re.MULTILINE)
    for c in categories:
        
        #### Category Name
        category = find(r'(?<=Block )"(\w+)"', c)
        root[category] = {}
        root[category]["Register"] = {}
        
        #### Category Address
        root[category]["Address"] = find(r'(?<=\nAddress: )(\w+)', c)
        
        #### Category Size
        root[category]["Size"] = int(find(r'(?<=\nSize:)\s+(\w+)', c))

        print("[Cateroty]", category)
        parse_registers(root[category]["Register"], c)


with open(input_file, 'r') as f:
    data = f.read()
    
    root = {}
    
    parse_categories(root, data)
    json_data = json.dumps(root, indent=2)

    print("Register Count: ", register_count)
    with open(output_file, "w") as f:
        f.write(json_data)

print("Done")

[Cateroty] CLKRST
	[Register] CLKRST_FRO_CTL
		Bit:  FRO_FTRIM_COARSE [12, 16]
		Bit:  FRO_FTRIM_FINE [8, 11]
		Bit:  FRO_HW_EN [0]
	[Register] CLKRST_LSO_TRIM
		Bit:  LSO_FREQ_ADJ [0, 11]
	[Register] SYSCLK_SYS_CTRL_0
		Bit:  SYSCLK_PLL_RST [12]
		Bit:  REFCLK_PD [0, 7]
	[Register] SYSCLK_SYS_CTRL_1
		Bit:  REFCLK_ABE_TX1_DIVSEL [24, 30]
		Bit:  REFCLK_ABE_TX0_DIVSEL [16, 22]
		Bit:  REFCLK_USB_DIVSEL [8, 14]
		Bit:  REFCLK_DP_RX_DIVSEL [0, 6]
	[Register] SYSCLK_SYS_CTRL_2
		Bit:  REFCLK_TEST_DIVSEL [24, 30]
		Bit:  REFCLK_SYS_DIVSEL [16, 22]
		Bit:  REFCLK_VID_TX1_DIVSEL [8, 14]
		Bit:  REFCLK_VID_TX0_DIVSEL [0, 6]
	[Register] SYSCLK_SYS_CTRL_3
		Bit:  XOSC_RETCIN [7]
		Bit:  XOSC_GMSEL [3, 6]
		Bit:  XOSC_INTERNAL_RES [2]
		Bit:  XOSC_BYPASS [1]
		Bit:  XOSC_EN [0]
	[Register] SYSCLK_DP_SYS_SSDSM_0
		Bit:  DP_SSDSM_LCK_RST [20]
		Bit:  DP_SSDSM_RESET [19]
		Bit:  DP_SSDSM_SSC_EN [18]
		Bit:  DP_SSDSM_OFFSET_HI [16, 17]
		Bit:  DP_SSDSM_DEV [12, 15]
		Bit:  DP_SSDSM_FREQ [0, 11]
	[Re